In [1]:
pip install tensorflow pandas librosa

^C
Note: you may need to restart the kernel to use updated packages.


                                              0.0/253.7 kB ? eta -:--:--
     -------------------------------------- 253.7/253.7 kB 7.9 MB/s eta 0:00:00
                                              0.0/284.2 MB ? eta -:--:--
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                             1.1/284.2 MB 34.0 MB/s eta 0:00:09
                                              1.4/284.2 MB 4.4 MB/s eta 0:01:05
                                              1.4/284.2 MB 4.4 MB/s eta 0:01:05
                                              1.4/284.2 MB 4.4 MB/s eta 0:01:05
                                              2.3/284.

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\Monkey.D.Luffy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "c:\Users\Monkey.D.Luffy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\req_command.py", line 248, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Monkey.D.Luffy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "c:\Users\Monkey.D.Luffy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "c:\Users\Monkey.D.Luffy\App

In [13]:
import pandas as pd
import numpy as np
import librosa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking, Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the metadata
metadata_path = 'svarah\meta_speaker_stats.csv'
metadata = pd.read_csv(metadata_path)

In [3]:
# Load audio and text data
audio_data = []
text_data = []

In [74]:
# import pandas as pd

# # Assuming df is your DataFrame
# df = pd.DataFrame({
#     'audio_filepath': metadata['audio_filepath'],
#     'text': metadata['text']
# })

# # Check if 'audio_filepath' column is empty
# audio_empty = df['audio_filepath'].isna() | df['audio_filepath'].eq('')

# # Check if 'text' column is empty
# text_empty = df['text'].isna() | df['text'].eq('')

# # Print the results
# print("Audio column is empty:")
# print(audio_empty)

# print("\nText column is empty:")
# print(text_empty)

In [4]:
for index, row in metadata.iterrows():
    audio_path = row['audio_filepath']

    # Load audio file using librosa
    audio, _ = librosa.load('svarah/'+audio_path, sr=16000)  # Adjust the sampling rate if necessary
    audio_data.append(audio)
    
    # Load text
    """ with open(text_path, 'r') as file:
        text = file.read() """
    text_data.append(row['text'])

In [5]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [6]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(text_data)
print(len(sequences))
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
print(len(padded_sequences))

6656
74
6656


In [7]:
# Create input data
print('Length : ',len(audio_data))
X = np.array(audio_data)
y = padded_sequences
print(len(X),len(y))

Length :  6656
6656 6656


C:\Users\Monkey.D.Luffy\AppData\Local\Temp\ipykernel_38520\2881343204.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(audio_data)


In [8]:
# Pad sequences
X_padded = pad_sequences(X, dtype='float32', padding='post', truncating='post') 

In [70]:
""" print("Original shape of X:", X.shape)

# Reshape X to add the time dimension
X = X.reshape((X.shape[0], 1))

print("New shape of X:", X.shape) """
# Convert to TensorFlow tensors
# X_padded_tf = tf.convert_to_tensor(X_padded)
# y_tf = tf.convert_to_tensor(y)

' print("Original shape of X:", X.shape)\n\n# Reshape X to add the time dimension\nX = X.reshape((X.shape[0], 1))\n\nprint("New shape of X:", X.shape) '

In [9]:
# Convert tensor indices to NumPy arrays
# X_padded_np = np.array(X_padded_tf)
# y_np = np.array(y_tf)
# Assuming X_padded and y are your padded sequences and labels
X_padded_np = np.array(X_padded)
y_np = np.array(y)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded_np, y_np, test_size=0.2, random_state=42)

In [11]:
# Build the model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(X_train.shape[1],1)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dense(total_words, activation='softmax'))

In [22]:
# Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Compile the model with sparse categorical crossentropy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model without one-hot encoding
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))
# model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test_reshaped, y_test))


MemoryError: Unable to allocate 9.48 GiB for an array with shape (5324, 477937) and data type float32

In [1]:
def data_generator(X, y, batch_size):
    while True:
        indices = np.random.randint(0, len(X), batch_size)
        yield X[indices], y[indices]

model.fit(data_generator(X_train, y_train, 32), steps_per_epoch=len(X_train)//32, epochs=10, validation_data=(X_test_reshaped, y_test))


NameError: name 'model' is not defined

In [15]:
def sequence_generator(X, y, batch_size):
    while True:
        indices = np.random.choice(len(X), batch_size)
        X_batch = X[indices]
        y_batch = y[indices]
        yield X_batch, y_batch


In [20]:
# Convert labels to one-hot encoded format
y_train_one_hot = to_categorical(y_train, num_classes=total_words)
y_test_one_hot = to_categorical(y_test, num_classes=total_words)

# Reshape X_test
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

MemoryError: Unable to allocate 12.6 GiB for an array with shape (393976, 8600) and data type float32

In [18]:
batch_size = 32  # You can adjust this based on your memory constraints
train_generator = sequence_generator(X_train, y_train, batch_size)

model.fit(train_generator, steps_per_epoch=len(X_train)//batch_size, epochs=10, validation_data=(X_test_reshaped, y_test_one_hot))


NameError: name 'y_test_one_hot' is not defined